In [25]:
import requests as req
import json
import math
import os

import matplotlib.pyplot as plt
import seaborn as sns

# Get metadata from Dataverse for each DOI

In [26]:
from string import Template
query_base=Template('https://dataverse.harvard.edu/api/search?q="${doi}"&show_facets=true')

In [27]:
import pandas
unique_dois = pandas.read_csv("../get-dois/dataset_dois.txt", names=['DOI'])

In [28]:
unique_dois.head()

,DOI
0,doi:10.7910/DVN/U3QJQZ
1,doi:10.7910/DVN/Z7H44N
2,doi:10.7910/DVN/HRLHA4
3,doi:10.7910/DVN/RJWU7A
4,doi:10.7910/DVN/FAAMAX


In [29]:
unique_dois = unique_dois['DOI'].values.tolist()

In [30]:
len(unique_dois)

2170

In [31]:
query = query_base.substitute(doi = unique_dois[0])
res = req.get(query)
res_dict = json.loads(res.text)

In [32]:
time = str(res_dict['data']['items'][0]['published_at'][:4])
time

'2020'

In [36]:
all_metadata=""
all_metadata = ','.join(["DOI","publicationDate","publisher","subject"]) + os.linesep
for i,doi in enumerate(unique_dois):
        query = query_base.substitute(doi = doi)

        res = req.get(query)
        res_dict = json.loads(res.text)
        try:
            time = res_dict['data']['items'][0]['published_at'][:4]
        except IndexError:
            print res_dict['data']['items']
            
        try:
            subs = res_dict['data']['facets'][0]['subject_ss']['labels']
            subject = ";".join(str(s.keys()[0]) for s in subs)
        except KeyError:
            subs = "NA"
        
        try:
            publisher = res_dict['data']['items'][0]['identifier_of_dataverse']
        except:
            print doi
            print res_dict['data']
            
        metadata_line = ','.join([doi.strip(), time, publisher.replace(",",";"), subject.replace(",",";")])
        #print metadata_line
        all_metadata += metadata_line + os.linesep

doi:10.7910/DVN/Y3VPIG
{u'spelling_alternatives': {}, u'total_count': 2, u'facets': [{u'fileTypeGroupFacet': {u'labels': [{u'Tabular Data': 1}], u'friendly': u'File Type'}, u'authorAffiliation_ss': {u'labels': [{u'ETH Zurich': 1}, {u'FORTH and University of Crete': 1}, {u'University of Konstanz': 1}, {u'University of Konstanz /  University of Santiago de Compostela': 1}], u'friendly': u'Author Affiliation'}, u'publicationDate': {u'labels': [{u'2016': 1}, {u'2020': 1}], u'friendly': u'Publication Year'}, u'subject_ss': {u'labels': [{u'Computer and Information Science': 1}, {u'Social Sciences': 1}], u'friendly': u'Subject'}, u'authorName_ss': {u'labels': [{u'Benitez-Baleato, Suso': 1}, {u'Dimitropoulos, Xenofontas': 1}, {u'Glatz, Eduard': 1}, {u'Hunziker, Philipp': 1}, {u'Weidmann, Nils': 1}], u'friendly': u'Author Name'}, u'fileAccess': {u'labels': [{u'Public': 1}], u'friendly': u'Access'}, u'fileTag': {u'labels': [{u'Data': 1}], u'friendly': u'File Tag'}}], u'q': u'"doi:10.7910/DVN/Y3V

In [34]:
import pandas as pd 
from StringIO import StringIO

df = pd.read_csv(StringIO(all_metadata))
#df = pd.read_csv(StringIO(all_metadata), delimiter=',')
df.head()

,DOI,publicationDate,publisher,subject
0,doi:10.7910/DVN/U3QJQZ,2020,harvard,Medicine; Health and Life Sciences
1,doi:10.7910/DVN/Z7H44N,2017,PSRM,Social Sciences
2,doi:10.7910/DVN/HRLHA4,2016,pan,Mathematical Sciences;Social Sciences
3,doi:10.7910/DVN/RJWU7A,2015,intertransferuschina,Social Sciences
4,doi:10.7910/DVN/FAAMAX,2018,BLS-PNAS,Social Sciences


In [35]:
len(df)

285

In [ ]:
with open('all_metadata_sept.txt','w') as f:
    f.write(all_metadata.encode('utf8'))

# Get publisher metadata from Dataverse

In [ ]:
import pandas as pd
metadata = pd.read_csv("all_metadata_sept.txt")
metadata.head()

In [ ]:
unique = metadata['publisher'].unique()
unique[:10]

In [ ]:
from string import Template
import requests as req
import json
import os

qbase=Template('https://dataverse.harvard.edu/api/dataverses/${iddv}')

all_journals = ""
for i,iddv in enumerate(unique):
    query = qbase.substitute(iddv = iddv)
    res = req.get(query)
    res_dict = json.loads(res.text)
    
    try:
        name = res_dict['data']['name']
        dvtype = res_dict['data']['dataverseType']
        
        metadata_line = '\t'.join([iddv.strip(), dvtype, name])
        #print metadata_line
        all_journals += metadata_line + os.linesep
    except:
        print res_dict

In [ ]:
from StringIO import StringIO
dj = pd.read_csv(StringIO(all_journals), sep='\t', names=['publisher', 'type', 'name'])

dj.head()

In [ ]:
len(metadata)

In [ ]:
len(dj)

In [ ]:
merged = pd.merge(metadata, dj, on='publisher')

In [ ]:
len(merged)

In [ ]:
merged.head()

In [ ]:
merged.to_csv('metadata_merged.csv')